In [ ]:
!pip install soundfile

In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import soundfile as sf
import numpy as np
import os

# Function to remove dead noise from the beginning and end of a multi-channel audio signal
def remove_dead_noise(audio, threshold=0.01):
    rms = np.sqrt(np.mean(np.square(audio), axis=-1))
    start_idx = next((i for i, x in enumerate(rms) if x > threshold), None)
    end_idx = next((i for i, x in enumerate(reversed(rms)) if x > threshold), None)
    if end_idx is not None:
        end_idx = len(rms) - end_idx
    return audio[start_idx:end_idx]

# Function to handle folder processing
def process_folder(_):
    folder_path = folder_text.value
    
    if not os.path.exists(folder_path):
        print("The specified folder does not exist.")
        return
    
    # Create a sub-folder to save edited files
    output_folder = os.path.join(folder_path, 'edited_audio_files')
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Get list of all audio files in the folder
    audio_files = [f for f in os.listdir(folder_path) if f.endswith(('.wav', '.flac', '.ogg'))]
    
    # Initialize progress bar
    num_files = len(audio_files)
    progress = widgets.IntProgress(value=0, min=0, max=num_files, description='Processing:')
    display(progress)
    
    for i, name in enumerate(audio_files):
        file_path = os.path.join(folder_path, name)
        
        # Read the audio file
        audio, samplerate = sf.read(file_path)
        
        # Process the audio to remove dead noise
        edited_audio = remove_dead_noise(audio)
        
        # Save the edited audio
        edited_filename = os.path.join(output_folder, f"{name}")
        sf.write(edited_filename, edited_audio, samplerate)
        
        # Update progress bar
        progress.value = i + 1
    
    # Clear progress bar and display completion message
    clear_output(wait=True)
    print(f"{num_files} audio files processed and saved in the '{output_folder}' folder.")

# Create a Text widget for folder path input
folder_text = widgets.Text(
    value='',
    placeholder='Enter folder path',
    description='Folder:',
    disabled=False
)

# Create a Button widget to trigger processing
process_button = widgets.Button(
    description='Process Folder',
    disabled=False,
    button_style='',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to process all audio files in the folder'
)

# Display the widgets
display(folder_text, process_button)

# Attach the function to run when the button is clicked
process_button.on_click(process_folder)

280 audio files processed and saved in the 'C:\Users\Oskar\Downloads\wav-20230912T033905Z-001\wav\edited_audio_files' folder.
